# Analysis Chain Example

This analysis chain is an example of an **input/output** study. 

1. We draw a sample from the nifty prior distribution
2. generate simulations (using `gen_amp` + `halld_sim/Piecewise` amplitude mimicking prior sample)
3. fit the simulatisons using binned fits (by `AmpTools`) and `iftpwa`

We would like to show a high probability of reconstruction. If this occurs for a wide variety of samples (and we believe our data could be approximately described by the very-flexible nifty prior) then we have successfully demonstrated that inference would be robust. This is in constrast to standard analyses in the field where we come up with ad-hoc models (implicit priors) then perform an I/O study on it. In this case, perhaps only a handful of rigid models are tested which would not necessarily translate to robust inference.

```{note}
This analysis is just for illustration purposes, optimization was not performed for very long.
```

In [1]:
from pyamptools import atiSetup
import os
os.environ["PYTHONWARNINGS"] = "ignore"

Welcome to JupyROOT 6.28/06


In [3]:
def run_cmd_list(cmds):
    for command in cmds:
        print(f"running: {command}")
        os.system(command)

In [4]:
atiSetup.setup(globals(), use_fsroot=True, use_genamp=True)

atiSetup| node called python

------------------------------------------------
atiSetup| MPI is disabled
atiSetup| GPU is disabled
------------------------------------------------


atiSetup| Loading library libIUAmpTools.so ............  ON
atiSetup| Loading library libAmpTools.so ..............  ON
atiSetup| Loading library libAmpPlotter.so ............  ON
atiSetup| Loading library libAmpsDataIO.so ............  ON
atiSetup| Loading library libFSRoot.so ................  ON
atiSetup| Loading library libAmpsGen.so ...............  ON


------------------------------------------------
------------------------------------------------

atiSetup| Saved aliases found in /w/halld-scshelf2101/lng/WORK/PyAmpTools9/src/pyamptools/.aliases.txt, attempting to load...
atiSetup| minor warning: Unable to alias FSFitFunctions - doesn't exist under ROOT namespace
atiSetup| minor warning: Unable to alias FSFitPrivate - doesn't exist under ROOT namespace
atiSetup| minor warning: Unable to alias omegap

(False, False, 0)

In [5]:
PYAMPTOOLS_HOME = os.getenv("PYAMPTOOLS_HOME")
subdir = f"{PYAMPTOOLS_HOME}/demos/ift_example/"
print(f"subdir: {subdir}")

subdir: /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example/


In [4]:
# cleanup
os.system(f'rm -rf {subdir}/NiftyFits')
os.system(f'rm -rf {subdir}/AmpToolsFits')
os.system(f'mkdir -p {subdir}/NiftyFits/prior_sim_DATA')


0

## Draw prior sample from `iftpwa`

In [5]:
# Draw from the nifty prior distribution
nifty_prior_draw_cmds = [
    f'sed -i -E "s/force_load_normint: true/force_load_normint: false/" {subdir}/pyamptools.yaml',
    f'mkdir -p {subdir}/NiftyFits/prior_sim',
    f'pa run_ift {subdir}/pyamptools.yaml --prior_simulation',
    f'mv {subdir}/NiftyFits/niftypwa_fit.pkl {subdir}/NiftyFits/prior_sim/niftypwa_fit.pkl',
    f'rm -rf {subdir}/NiftyFits/prior_sim/plots; iftPwaPlot --fitResult {subdir}/NiftyFits/prior_sim/niftypwa_fit.pkl --plotFolder {subdir}/NiftyFits/prior_sim/plots --n_prior_samples 1000'
]

run_cmd_list(nifty_prior_draw_cmds)

running: sed -i -E "s/force_load_normint: true/force_load_normint: false/" /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: mkdir -p /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim
running: pa run_ift /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml --prior_simulation
running: mv /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/niftypwa_fit.pkl /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim/niftypwa_fit.pkl
running: rm -rf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim/plots; iftPwaPlot --fitResult /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim/niftypwa_fit.pkl --plotFolder /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim/plots --n_prior_samples 1000

Running command: python /w/halld-scshelf2101/lng/WORK/PyAmpTools9/bin/run_ift.py /w/ha

--------------------------------------------------------------------------
Open MPI failed an OFI Libfabric library call (fi_endpoint).  This is highly
unusual; your job may behave unpredictably (and/or abort) after this.

  Local host: ifarm2401
  Location: mtl_ofi_component.c:513
  Error: Invalid argument (22)
--------------------------------------------------------------------------
[ifarm2401.jlab.org:3863698] 40 more processes have sent help message help-mtl-ofi.txt / OFI call fail
[ifarm2401.jlab.org:3863698] Set MCA parameter "orte_base_help_aggregate" to 0 to see all help / error messages
15:52:40| GlueXManager| Number of Fit Parameters (2x number of waves): 4

15:52:40| GlueXManager| Exporting auxiliary information
Calculating total intensities ...: 2it [00:00, 13210.41it/s]
Plotting waves ...: 100%|██████████| 2/2 [00:00<00:00,  2.73it/s]


## Simulate data from the model and phase space MC

In [6]:
max_mass = 1.72
min_mass = 1.04
t_slope = 3.0
min_ebeam = 8.2
max_ebeam = 8.8
data_nsamples = 100000
gen_nsamples = 500000

generation_cmds = [
    f'python {subdir}/prior_sim_gen.py', # create the amptools cfg file using halld_sim/Piecewise to model the nifty prior sample
    
    f'pa gen_amp {subdir}/prior_sim_gen.cfg -o {subdir}/NiftyFits/prior_sim_DATA/data000.root -l {min_mass} -u {max_mass} -n {data_nsamples} -a {min_ebeam} -b {max_ebeam} -t {t_slope}',
    f'mv gen_amp_diagnostic.root {subdir}/NiftyFits/prior_sim_DATA/gen_amp_diagnostic_data.root 2>/dev/null',
    
    f'pa gen_amp {subdir}/prior_sim_gen.cfg -o {subdir}/NiftyFits/prior_sim_DATA/accmc000.root -l {min_mass} -u {max_mass} -n {gen_nsamples} -a {min_ebeam} -b {max_ebeam} -t {t_slope} -f ',
    f'cp {subdir}/NiftyFits/prior_sim_DATA/accmc000.root {subdir}/NiftyFits/prior_sim_DATA/genmc000.root', # uniform acceptance example
    f'mv gen_amp_diagnostic.root {subdir}/NiftyFits/prior_sim_DATA/gen_amp_diagnostic_ps.root 2>/dev/null',
]

run_cmd_list(generation_cmds)

running: python /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//prior_sim_gen.py
running: pa gen_amp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//prior_sim_gen.cfg -o /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim_DATA/data000.root -l 1.04 -u 1.72 -n 100000 -a 8.2 -b 8.8 -t 3.0
running: mv gen_amp_diagnostic.root /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim_DATA/gen_amp_diagnostic_data.root 2>/dev/null
running: pa gen_amp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//prior_sim_gen.cfg -o /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim_DATA/accmc000.root -l 1.04 -u 1.72 -n 500000 -a 8.2 -b 8.8 -t 3.0 -f 
running: cp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim_DATA/accmc000.root /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/prior_sim_DATA/genmc000.root
running: mv gen_amp_diagnostic

/w/halld-scshelf2101/lng/micromamba/envs/pyamptools/lib/python3.9/site-packages/nifty8/library/correlated_fields.py:460: UserWarning: unable to add JAX operator for total_N=4
  warn(f"unable to add JAX operator for total_N={total_N}")


## Run the MLE fits using `AmpTools`

In [7]:
mle_cmds = [
    f'pa run_cfgGen {subdir}/pyamptools.yaml',
    f'pa run_divideData {subdir}/pyamptools.yaml',
    f'pa run_mle {subdir}/pyamptools.yaml',
    f'pa run_processEvents {subdir}/pyamptools.yaml',
]

run_cmd_list(mle_cmds)

running: pa run_cfgGen /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: pa run_divideData /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: pa run_mle /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: pa run_processEvents /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml --verbose

Running command: python /w/halld-scshelf2101/lng/WORK/PyAmpTools9/bin/run_cfgGen.py /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml


---------------------
Running /w/halld-scshelf2101/lng/WORK/PyAmpTools9/bin/run_cfgGen.py
  yaml location: /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
---------------------

Particles in reaction:  ['Beam', 'Proton', 'Pi0', 'Eta']
Using real waves: ['']
Using fixed waves: ['']
No background file found. AmpTools will assume data file contains pure signal
atiSetup| pa called python

-

## Run the `iftpwa` fits

In [24]:
ift_cmds = [
    f'sed -i -E "s/force_load_normint: false/force_load_normint: true/" {subdir}/pyamptools.yaml',
    f'rm -rf {subdir}/NiftyFits/.nifty_expanded.yaml', # trash previous expanded yaml which might not have the right IFT_MODEL.scale since \'auto\' could be overwritten
    f'pa run_ift {subdir}/pyamptools.yaml',
    f"rm -rf {subdir}/NiftyFits/plots",
    f"iftPwaPlot --fitResult {subdir}/NiftyFits/niftypwa_fit.pkl --plotFolder {subdir}/NiftyFits/plots  --massIndepFit {subdir}/NiftyFits/.nifty_expanded.yaml --n_prior_samples 1000",
    f"plotConvergence --inputFolder {subdir}/NiftyFits/fit_parameter_history --plotFolder {subdir}/NiftyFits/convergence_plots",
]

run_cmd_list(ift_cmds)

running: sed -i -E "s/force_load_normint: false/force_load_normint: true/" /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: rm -rf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/.nifty_expanded.yaml
running: pa run_ift /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
running: rm -rf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots
running: iftPwaPlot --fitResult /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/niftypwa_fit.pkl --plotFolder /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots  --massIndepFit /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/.nifty_expanded.yaml --n_prior_samples 1000
running: plotConvergence --inputFolder /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/fit_parameter_history --plotFolder /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//Nif

17:21:27| GlueXManager| Number of Fit Parameters (2x number of waves): 4

Sample for nBar calibration: 100%|██████████| 200/200 [00:05<00:00, 37.38it/s] 
17:21:33| GlueXManager| 4 Amplitude Terms: ['PosRe.Sp0+', 'PosRe.Dp2+', 'PosIm.Sp0+', 'PosIm.Dp2+']
17:21:33| GlueXManager| ------ Checking data shapes ------
17:21:33| GlueXManager| nTerms: 4
17:21:33| GlueXManager| nmbReactions: 1
17:21:33| GlueXManager| nmbMasses: 56
17:21:33| GlueXManager| nmbTprimes: 1
17:21:33| - - - - - - - - - - - - - - - - - - 
17:21:33| GlueXManager| Shape of reactions: (1, 56, 1) ~ (2*nTerms+[weight, bin], nEvents)
17:21:33| GlueXManager| Shape of nGens: (1, 56, 1) ~ (2*nTerms+[weight, bin], nEvents)
17:21:33| GlueXManager| Shape of nAccs: (1, 56, 1) ~ (2*nTerms+[weight, bin], nEvents)
17:21:33| GlueXManager| Shape of ampIntss: (1, 56, 1, 4, 4) ~ (nmbReactions, nmbMasses, nmbTprimes, nTerms, nTerms)
17:21:33| GlueXManager| Shape of normIntss: (1, 56, 1, 4, 4) ~ (nmbReactions, nmbMasses, nmbTprimes, nTerms, 

## Plot the moments and dump all results to a csv file

In [25]:
aux_cmds = [
    f'pa run_momentPlotter {subdir}/pyamptools.yaml -n 10',
    f'pa run_resultDump {subdir}/pyamptools.yaml -n 10',
]

run_cmd_list(aux_cmds)

running: pa run_momentPlotter /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml -n 10
running: pa run_resultDump /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml -n 10

Running command: python /w/halld-scshelf2101/lng/WORK/PyAmpTools9/bin/run_momentPlotter.py /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml -n 10


---------------------
Running /w/halld-scshelf2101/lng/WORK/PyAmpTools9/bin/run_momentPlotter.py
  yaml location: /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//pyamptools.yaml
---------------------

io| Attempting to load AmpTools results...
io| Searching for amptools files using this pattern: /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//AmpToolsFits//group_*/bin_[].cfg
io| Remaining number of AmpTools fits (across randomizations and kinematic bins) after applying mle_query_1 = 'status == 0 & ematrix == 3': 150
io| Calculating delta_nll...
io| Remaining number of

--------------------------------------------------------------------------

  Local host:   ifarm2401
  Local device: mlx5_1
--------------------------------------------------------------------------
--------------------------------------------------------------------------

  Local host:   ifarm2401
  Local device: mlx5_1
--------------------------------------------------------------------------
--------------------------------------------------------------------------
Open MPI failed an OFI Libfabric library call (fi_endpoint).  This is highly
unusual; your job may behave unpredictably (and/or abort) after this.

  Local host: ifarm2401
  Location: mtl_ofi_component.c:513
  Error: Invalid argument (22)
--------------------------------------------------------------------------


## Save copies for the gallery

In [7]:
# # copy some select plots to be used in the gallery and perform cleanup
# copy_cmds = [
#     f'cp {subdir}/NiftyFits/plots/moments/H0\(0,0\)_t0.5.pdf {PYAMPTOOLS_HOME}/assets/images/H0_0_0.pdf',
#     f'cp {subdir}/NiftyFits/plots/moments/H0\(2,0\)_t0.5.pdf {PYAMPTOOLS_HOME}/assets/images/H0_2_0.pdf',
#     f'cp "{subdir}/NiftyFits/plots/parameters/m_{{a_2(1320)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a2_1320_kde.pdf"',
#     f'cp "{subdir}/NiftyFits/plots/parameters/m_{{a_0(980)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a0_980_kde.pdf"',
#     f'cp "{subdir}/NiftyFits/convergence_plots/m_{{a_2(1320)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a2_1320_convergence.pdf"',
#     f'cp "{subdir}/NiftyFits/convergence_plots/m_{{a_0(980)}}.pdf" "{PYAMPTOOLS_HOME}/assets/images/m_a0_980_convergence.pdf"',
# ]

# run_cmd_list(copy_cmds)

# # Copy all pdfs to assets/images and convert to png to be rendered in gallery page
# os.chdir(f'{PYAMPTOOLS_HOME}/assets/images')
# os.system('for f in *.pdf; do convert -density 300 "$f" "${f%.pdf}.png"; done')
# os.system('rm *.pdf')

running: cp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots/intensity_and_phase/intensity_and_phase_Dp2+_tprimeBinID_0_tprime_0.0-1.0.pdf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/assets/images/Dp2+.pdf
running: cp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots/intensity_and_phase/intensity_and_phase_Sp0+_tprimeBinID_0_tprime_0.0-1.0.pdf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/assets/images/Sp0+.pdf
running: cp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots/moments/H0\(0,0\)_t0.5.pdf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/assets/images/H0_0_0.pdf
running: cp /w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots/moments/H0\(2,0\)_t0.5.pdf /w/halld-scshelf2101/lng/WORK/PyAmpTools9/assets/images/H0_2_0.pdf
running: cp "/w/halld-scshelf2101/lng/WORK/PyAmpTools9/demos/ift_example//NiftyFits/plots/parameters/m_{a_2(1320)}.pdf" "/w/halld-scshelf2101/lng/WORK/PyAmpTools9/assets/

0

## Cleanup

In [35]:
# clean up 
os.system('rm *.pdf')
os.system(f'rm -rf {subdir}/AmpToolsFits')
os.system(f'rm -rf {subdir}/NiftyFits')
os.system(f'rm -rf {subdir}/*pkl')
os.system(f'rm -rf {subdir}/*cfg')
os.system(f'rm -rf {subdir}/*csv')

0